In [1]:
import pandas as pd

In [2]:
df_raw = pd.DataFrame(columns=["sequence", "class"])      

with open("../data/raw/anti_cp/anticp2_alternate_internal_negative.txt") as reader:
    for line in reader:
        df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)

with open("../data/raw/anti_cp/anticp2_alternate_internal_positive.txt") as reader:
    for line in reader:
        df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)

with open("../data/raw/anti_cp/anticp2_alternate_validation_negative.txt") as reader:
    for line in reader:
        df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)

with open("../data/raw/anti_cp/anticp2_alternate_validation_positive.txt") as reader:
    for line in reader:
        df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)


/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 0}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

/tmp/ipykernel_830142/3106872347.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": line.strip("\n"), "class": 1}, ignore_index = True)
/tmp/ipykernel_830142/3106872347.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append({"sequence": 

In [3]:
import itertools

aas = 'ACDEFGHIKLMNPQRSTVWY'

def compute_di_aa_composition(sequence):
    
    di_aas = sorted([''.join(di_aa) for di_aa in set(itertools.product(aas, aas))])
    di_aas_count = {di_aa:sequence.count(di_aa) for di_aa in di_aas}
    di_aas_count = {di_aa:di_aas_count[di_aa] / sum(di_aas_count.values()) for di_aa in di_aas}
    return di_aas_count

In [4]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np

def compute_sequence_data(sequence):
    
    sequence = sequence.upper()
    protein_analysis = ProteinAnalysis(sequence)
    data = compute_di_aa_composition(sequence)    
    data['molecular_weight'] = protein_analysis.molecular_weight()
    data['aromaticity'] = protein_analysis.aromaticity()
    data['instability_index'] = protein_analysis.instability_index()
    data['gravy'] = protein_analysis.gravy()
    data['isoelectric_point'] = protein_analysis.isoelectric_point()
    data['length'] = len(sequence)
    return data

In [5]:
df_aa = pd.DataFrame([
    compute_sequence_data(sequence) for sequence in df_raw['sequence']
])

df_aa['class'] = df_raw['class']

In [6]:
df_aa

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YV,YW,YY,molecular_weight,aromaticity,instability_index,gravy,isoelectric_point,length,class
0,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,1420.6508,0.181818,12.590909,-0.418182,6.754200,11,0
1,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,3976.2909,0.027778,65.080556,-1.566667,6.387816,36,0
2,0.043478,0.00000,0.00000,0.000000,0.0,0.00000,0.021739,0.065217,0.00,0.021739,...,0.0,0.0,0.0,4652.5645,0.063830,22.725532,1.385106,8.754823,47,0
3,0.038462,0.00000,0.00000,0.038462,0.0,0.00000,0.000000,0.000000,0.00,0.038462,...,0.0,0.0,0.0,2892.2645,0.000000,73.818519,-0.181481,4.652238,27,0
4,0.085714,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,3888.3608,0.054054,72.559459,0.513514,4.501898,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.25,0.000000,...,0.0,0.0,0.0,1471.9147,0.076923,-10.361538,0.300000,10.602487,13,1
1936,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.333333,...,0.0,0.0,0.0,1102.4125,0.100000,-7.980000,0.790000,10.302064,10,1
1937,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,1643.0651,0.062500,-11.850000,1.393750,10.302064,16,1
1938,0.000000,0.03125,0.03125,0.000000,0.0,0.03125,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,3468.0556,0.030303,24.975758,0.296970,8.526347,33,1


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = df_aa.drop(['class'], axis=1)
y = df_aa['class'].astype(bool)

In [9]:
X

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YT,YV,YW,YY,molecular_weight,aromaticity,instability_index,gravy,isoelectric_point,length
0,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.0,1420.6508,0.181818,12.590909,-0.418182,6.754200,11
1,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.0,3976.2909,0.027778,65.080556,-1.566667,6.387816,36
2,0.043478,0.00000,0.00000,0.000000,0.0,0.00000,0.021739,0.065217,0.00,0.021739,...,0.0,0.0,0.0,0.0,4652.5645,0.063830,22.725532,1.385106,8.754823,47
3,0.038462,0.00000,0.00000,0.038462,0.0,0.00000,0.000000,0.000000,0.00,0.038462,...,0.0,0.0,0.0,0.0,2892.2645,0.000000,73.818519,-0.181481,4.652238,27
4,0.085714,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.0,3888.3608,0.054054,72.559459,0.513514,4.501898,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.25,0.000000,...,0.0,0.0,0.0,0.0,1471.9147,0.076923,-10.361538,0.300000,10.602487,13
1936,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.333333,...,0.0,0.0,0.0,0.0,1102.4125,0.100000,-7.980000,0.790000,10.302064,10
1937,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.0,1643.0651,0.062500,-11.850000,1.393750,10.302064,16
1938,0.000000,0.03125,0.03125,0.000000,0.0,0.03125,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.0,3468.0556,0.030303,24.975758,0.296970,8.526347,33


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [21]:
from sklearn.metrics import classification_report

y_pred = automl.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       False       0.94      0.98      0.96       225
        True       0.98      0.95      0.96       260

    accuracy                           0.96       485
   macro avg       0.96      0.96      0.96       485
weighted avg       0.96      0.96      0.96       485



In [22]:
def predict(sequence):
    X = pd.DataFrame([compute_sequence_data(sequence)])
    return automl.predict_proba(X)[0][1]

predict('kkvgtaakllsallkifkglvnaldnisirkqsarhypgsgiewrgypwr')

0.003973081260452956

In [ ]:
%%time

from tqdm.notebook import tqdm
from multiprocessing import Pool
import warnings
import random  

warnings.simplefilter("ignore")

top_peptides = []
top_k        = 1000

best_scores = []
best_score  = None

def generate_random_sequence(*args):
    
    random_length = random.randrange(5, 60)
    random_sequence = ''.join([random.choice(aas) for i in range(random_length)])
    anticp_proba = predict(random_sequence)
    return random_sequence, anticp_proba
    
for i, (random_sequence, anticp_proba) in enumerate(map(generate_random_sequence, tqdm(range(1_000_000)))):   
    
    top_peptides.append((random_sequence, anticp_proba))
    top_peptides = sorted(top_peptides, key=lambda x: x[1])[::-1][0:top_k]
    
    if best_score is None:
        best_score = anticp_proba
    elif anticp_proba > best_score:
        best_score = anticp_proba
        best_scores.append((i, best_score))

df_peptides = pd.DataFrame(top_peptides, columns=['peptide', 'proba'])
df_peptides

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [17]:
best_scores

[(2, 0.20097432006100183),
 (5, 0.9494651783887361),
 (105, 0.9859776515736489),
 (172, 0.9991152337863525)]

In [18]:
for r, row in df_peptides.iterrows():
    print(f'>{r}\n{row.peptide}')


KeyboardInterrupt

